# 01 - Extraction de données et visualisation
***

**Tutoriel :** Ce tutoriel fournit un guide étape par étape pour extraire des données du centre de données d'astronomie canadien (CADC) et du portail de données ouvertes de l'Agence spatiale canadienne. </br>

**Mission et Instrument** : [NEOSSat](https://www.asc-csa.gc.ca/eng/satellites/neossat/) <br>

**Objectifs de la Mission** :
* Connaissance de la situation spatiale : Objets géocroiseurs (astéroïdes, débris spatiaux)
* Cible astronomique : objets célestes (comètes, nébuleuses, galaxies, exoplanètes)

**Exigences du système** : Python 3.9 ou ultérieur <br>

**Niveau du tutoriel** : Élémentaire <br>

***
**Licence MIT** <br>
Copyright (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence spatiale canadienne, 2024. <br>
Droit d'auteur (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence Spatiale Canadienne, 2024.<br>

Pour plus d'informations, veuillez vous référer au fichier *License.txt*.

***
**Informations contextuelles** <br>
La mission [NEOSSat](https://www.asc-csa.gc.ca/eng/satellites/neossat/) lancée le 25 février 2013, est le premier télescope spatial au monde dédié à la détection et au suivi d'objets géocroiseurs.

Dans le cadre des efforts du Canada pour observer et suivre les astéroïdes, comètes, satellites et débris spatiaux, ce microsatellite, seulement de la taille d'une valise, est une plateforme d'observation puissante.

Opérant à une altitude d'environ 800 kilomètres, ce microsatellite orbite la Terre toutes les 100 minutes et fournit une surveillance continue des objets géocroiseurs. NEOSSat utilise une technologie avancée de roue de réaction, d'abord démontrée par le satellite [Microvariabilité et Oscillations des Étoiles (MOST)](https://www.asc-csa.gc.ca/eng/satellites/most/) lui permettant de balayer des régions de l'espace près du Soleil, aidant à identifier les astéroïdes et comètes qui peuvent passer près de la Terre.

Dans le cadre de l'engagement du Canada envers la sécurité spatiale, NEOSSat est impliqué dans le suivi des débris spatiaux, fournissant des données essentielles sur les objets qui sont difficiles à observer par les télescopes terrestres. La mission est financée conjointement par l'Agence spatiale canadienne (ASC) et Recherche et développement pour la défense Canada (RDDC). NEOSSat continue à faire progresser le rôle du Canada dans l'exploration spatiale et la sécurité grâce à ses contributions continues au suivi d'astéroïdes, à la surveillance de débris spatiaux, et à la recherche d'exoplanètes.

***
**Lecture complémentaire** <br>

Pour plus d'informations sur le télescope spatial NEOSSAT et les fichiers FITS, veuillez consulter le Guide de l'utilisateur d'image FITS NEOSSat via le lien suivant : https://donnees-data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/NEOSSAT/Supporting%20Documents/CSA-NEOSSAT-MAN-0002_FITS_IMAGE_UGUIDE-v4-00.pdf

**Extraction de données du portail de données ouvertes de l'Agence spatiale canadienne (ASC)** : L'Agence spatiale canadienne a une page dédiée aux ensembles de données NEOSSAT sur sa page web de données ouvertes et d'information qui peut être trouvée via le lien suivant : https://donnees-data.asc-csa.gc.ca/en/dataset/9ae3e718-8b6d-40b7-8aa4-858f00e84b30

**Extraction de données du Centre canadien de données astronomiques (CADC)** : CADC recommande l'installation du package CADCdata pour l'usage en Python. La documentation sur comment accéder à la bibliothèque est disponible avec le pydoc cadcdata. Pour plus d'informations, visitez : https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/doc/data/#library

## Option 1 - Extraction de données NEOSSAT de l'Agence spatiale canadienne (ASC)
Avec l'ensemble de données de l'Agence spatiale canadienne, vous pouvez filtrer les données NEOSSAT basées sur l'année, le jour de l'année et l'heure.

Pour comprendre la structure des données :

Les dossiers montrent l'Année. Les sous-dossiers montrent le jour de l'année. Les fichiers .fits sont nommés selon l'Année, le Jour de l'Année et l'Heure

Ex : NEOS_SCI_2021002000257.fits => [Année : 2021], [Jour de l'Année : 002], [Heure : 00:02:57]

### Importation des bibliothèques requises

In [ ]:
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
import matplotlib.pyplot as plt
from ftplib import FTP
import os
import re
from astropy.coordinates import SkyCoord
import astropy.units as u

La fonction ci-dessous filtrera à travers les données NEOSSAT de l'ASC basées sur l'année et le jour de l'année. Elle téléchargera ensuite les fichiers .fits vers le chemin local.

In [ ]:
def download_fits_files(year, day_of_year, local_directory):
    ftp_host = 'ftp.asc-csa.gc.ca'
    ftp = FTP (ftp_host)
    ftp.login ()

    ftp_directory = f'/users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/{year}/{day_of_year}/'
    ftp.cwd(ftp_directory)

    # List all .fits files in the directory
    fits_files = [file for file in ftp.nlst() if file.endswith('.fits')]

    for fits_file in fits_files:
        local_filename = os.path.join(local_directory, fits_file)
        with open(local_filename, 'wb') as f:
            ftp.retrbinary('RETR ' + fits_file, f.write)

    ftp.quit()
    print("Download complete.")

Sélectionnez l'année et le jour de l'année qui vous intéresse. Dans cet exemple, nous téléchargeons les données du 19 février 2024

In [ ]:
year = '2024'
day_of_year = '050'
local_directory = 'C:/Users/rsheikholmolouki/Desktop/NEOSSAT_CSA/Part 1/2024/050'
download_fits_files(year, day_of_year, local_directory)

Une autre façon de filtrer à travers les données en sélectionnant l'heure et seulement télécharger la gamme de fichiers .fits qui correspond aux critères d'heure. Nous commençons par définir la fonction :

In [ ]:
def download_fits_files_in_time_range(year, day_of_year, start_time, end_time, local_directory):
    ftp_host = 'ftp.asc-csa.gc.ca'
    ftp = FTP (ftp_host)
    ftp.login ()
    
    ftp_directory = f'/users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/{year}/{day_of_year}/'
    ftp.cwd(ftp_directory)
    
# List all .fits files in the directory
    fits_files = [file for file in ftp.nlst() if file.endswith('.fits')]

    start_time_str = start_time.zfill(6)
    end_time_str = end_time.zfill(6)

    filtered_files = []
    for fits_file in fits_files:
        match = re.search(r'(\d{6})\.fits$', fits_file)
        if match:
            file_time_str = match.group(1)
            if start_time_str <= file_time_str <= end_time_str:
                filtered_files.append(fits_file)

    if filtered_files:
        for fits_file in filtered_files:
            local_filename = os.path.join(local_directory, fits_file)
            with open(local_filename, 'wb') as f:
                ftp.retrbinary('RETR ' + fits_file, f.write)
        print("Download complete.")
        return filtered_files
    else:
        print("No files found in the specified time range.")
        return None

Sélectionnez l'année, le jour de l'année et la gamme de temps qui vous intéresse. Dans cet exemple, nous téléchargeons les données du 19 février 2024 de 00:02:57 à 01:09:20

In [ ]:
year = '2024'
day_of_year = '002'
start_time = '000000' #If the time in mind is 00:00:00 write 000000 to match the last 6 digits filename of the .fits file
end_time = '010257' #If the time in mind is 01:09:20 write 010920 to match the last 6 digits filename of the .fits file
local_directory = 'C:/Users/rsheikholmolouki/Desktop/NEOSSAT_CSA/Part 1/2024/time_range'

downloaded_files = download_fits_files_in_time_range(year, day_of_year, start_time, end_time, local_directory)

Visualisons la gamme sélectionnée des fichiers .fits.

In [ ]:
if downloaded_files:
    print("Downloaded files:", downloaded_files)
    # Visualize the .fits files
    from astropy.visualization import astropy_mpl_style
    from astropy.io import fits
    import matplotlib.pyplot as plt

    for fits_file in downloaded_files:
        fits_path = os.path.join(local_directory, fits_file)
        with fits.open(fits_path) as hdul:
            data = hdul[0].data

        plt.imshow(data, cmap='gray', interpolation='nearest') 
        plt.axis('off')  # Turn off axes
        plt.title(fits_file)
        plt.show()

## Option 2 - Extraction de données NEOSSAT du Centre canadien de données astronomiques (CADC)

Avec les ensembles de données CADC, en plus de filtrer les données basées sur le temps, vous pouvez aussi filtrer à travers les données NEOSSAT basées sur les coordonnées du ciel.

### Importation des bibliothèques requises

In [ ]:
from astroquery.cadc import Cadc
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Initialize the CADC client 
cadc = Cadc()

#Print information about available collection 
for collection, details in sorted(cadc.get_collections().items()):
   print(f'{collection} : {details}')

In [ ]:
#Define the target coordinates and search radius based on your preference
coords = SkyCoord(240, -30, unit = 'deg') #RA, DEC, Unit
radius = 2* u.deg

#Query CADC for data within the specificied region and collection 
results = cadc.query_region(coords,radius, collection = 'NEOSSAT')
print (results)

In [ ]:
#Filter the query results to select data with a specifc 'time_exposure'. 
# In this example 'time exposure' greater than 50 is selected

filtered_results = results[results['time_exposure'] > 50.0]

#Access data points from the filtered results 
print(filtered_results['time_exposure'][100])
print(filtered_results['position_dimension_naxis2'][100])
print(filtered_results['position_dimension_naxis1'][100])
print(filtered_results['instrument_keywords'][100])
print(filtered_results['metaRelease'][100])

In [ ]:
#Get a list of image URLs based on the filtered results
image_list = cadc.get_image_list(filtered_results, coords, radius)

#Print the number of images int he image_list 
print(len(image_list))

#print the last URL in the image_list 
print(image_list[-1])

#Get the filename of the 100th image from the image_list 
filename = image_list[100]
print(filename)

Visualisons le fichier .fits sélectionné.

In [ ]:
#Read the FITS image data from the file
image_data = fits.getdata(filename, ext =0)

In [ ]:
#Show the image file
plt.style.use(astropy_mpl_style)
plt.figure()
plt.axis ('off')
plt.imshow(image_data, cmap='gray')
plt.show()

Pour continuer avec le traitement des images, veuillez consulter le notebook suivant : https://github.com/asc-csa/NEOSSAT_Tutorial/blob/main/02_Pre%20Processing%20and%20Analysis.ipynb